#### Vector Stores and retrievers
We will learn about Langchain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data -- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

##### Documents
Langchain implements a document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

page_content: a string representing the content;

metadata:a dict containing arbitrary metadata.

The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual document object often represents a chunk of a larger document

In [1]:
from langchain_core.documents import Document
documents=[
    Document(
        page_content="dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="cats are independent pets that often enjoy their own space.",
        metadata={"source":"mammal-pets-doc"},
    ),
    
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech",
        metadata={"source":"bird-pets-doc"},
    ),
    Document(
        page_content="Raabits are social animals that need plenty of space to hop around.",
        metadata={"source":"mammal-pets-doc"},
    ),

]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Raabits are social animals that need plenty of space to hop around.')]

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
llm=ChatGroq(groq_api_key=groq_api_key,model="llama-3.3-70b-versatile")
llm



ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000191A1126EC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000191A1126AD0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
#VectorStores
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(documents,embedding=embeddings)
#all documenets will be converted to vectors using this embedding techniques and then it is going to get stored in ChromaDB
vectorstore

In [8]:
vectorstore.similarity_search("cat")

[Document(id='2e7afd9d-9d74-44e4-8ece-a7ef8f56ca3b', metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(id='87b92a5a-f696-4763-a9cd-3b59923e5ab2', metadata={'source': 'mammal-pets-doc'}, page_content='dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='59ff2e80-bfc5-42b5-afcc-dbb471cc423f', metadata={'source': 'mammal-pets-doc'}, page_content='Raabits are social animals that need plenty of space to hop around.'),
 Document(id='8f811a38-fa9e-4d7b-9ebf-f53fe5aaf3c8', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech')]

In [9]:
#Async Query
await vectorstore.asimilarity_search("cat")

[Document(id='2e7afd9d-9d74-44e4-8ece-a7ef8f56ca3b', metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(id='87b92a5a-f696-4763-a9cd-3b59923e5ab2', metadata={'source': 'mammal-pets-doc'}, page_content='dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='59ff2e80-bfc5-42b5-afcc-dbb471cc423f', metadata={'source': 'mammal-pets-doc'}, page_content='Raabits are social animals that need plenty of space to hop around.'),
 Document(id='8f811a38-fa9e-4d7b-9ebf-f53fe5aaf3c8', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech')]

In [10]:
#what if i need the scores as to how much similar is the content
vectorstore.similarity_search_with_score("cat")

[(Document(id='2e7afd9d-9d74-44e4-8ece-a7ef8f56ca3b', metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
  0.9351058006286621),
 (Document(id='87b92a5a-f696-4763-a9cd-3b59923e5ab2', metadata={'source': 'mammal-pets-doc'}, page_content='dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(id='59ff2e80-bfc5-42b5-afcc-dbb471cc423f', metadata={'source': 'mammal-pets-doc'}, page_content='Raabits are social animals that need plenty of space to hop around.'),
  1.5995341539382935),
 (Document(id='8f811a38-fa9e-4d7b-9ebf-f53fe5aaf3c8', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
  1.6284857988357544)]

##### Retrievers
Langchain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language Chains.

Langchain retrievers are Runnables, so they implement a standard set of methods(eg. synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains. 

We can create a simple version ourselves, without subclassing retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily

In [12]:
#1st technique
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
#using runnable lamda we are able to convert vectorstore into retriever
retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1) #k=1 means that give me the topmost result
retriever.batch(["cat","dog"]) #the runnable lambda function is going to get applied to all the parameters we will give in the batch


[[Document(id='2e7afd9d-9d74-44e4-8ece-a7ef8f56ca3b', metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')],
 [Document(id='87b92a5a-f696-4763-a9cd-3b59923e5ab2', metadata={'source': 'mammal-pets-doc'}, page_content='dogs are great companions, known for their loyalty and friendliness.')]]

VectorStores implement an as_retriever method that will generate a retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. 

In [ ]:
#this retriever acts as an interface to pass any info to the vectorstore and give any response
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
) 
retriever.batch(["cat","dog"])

[[Document(id='2e7afd9d-9d74-44e4-8ece-a7ef8f56ca3b', metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')],
 [Document(id='87b92a5a-f696-4763-a9cd-3b59923e5ab2', metadata={'source': 'mammal-pets-doc'}, page_content='dogs are great companions, known for their loyalty and friendliness.')]]

In [ ]:
#RAG
#integration of retriever with chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message=""" 
Answer this question using the provided context only.
{question}
Context:
{context}"""
prompt=ChatPromptTemplate.from_messages([("human",message)])
#how to pass retriever in the chain as now it is a subclass of runnable
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm
#this means i will get the context from the retriever, the question i will get when i invoke the runnablepassthrough method

In [17]:
response=rag_chain.invoke("tell me about dogs")
print(response.content)

According to the provided context, dogs are great companions, known for their loyalty and friendliness.
